In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#read data
data=pd.read_json('archive (4)/Sarcasm_Headlines_Dataset_v2.json',lines=True)

In [3]:
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [4]:
data.is_sarcastic.value_counts()

0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [6]:
#splitting data into train and test
X_train, X_test, Y_train, Y_test= train_test_split(data.headline,data.is_sarcastic,test_size=0.2)

In [5]:
#load bert model from tensorflow
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/2")

In [7]:
#get tokenized keys
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [13]:
#pre process data
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
X=outputs['pooled_output']

#required layers for the model
nn= tf.keras.layers.Dense(128,activation='relu')(X)
nn = tf.keras.layers.Dropout(0.3, name="dropout")(nn)
nn = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(nn)


#making the model
model = tf.keras.Model(inputs=[text_input], outputs = [nn])

In [14]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  52779265    keras_layer[1][0]                
                                                                 keras_layer[1][1]                
                                                                 keras_layer[1][2]                
____________________________________________________________________________________________

In [18]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [19]:
checkpoint_filepath = 'model/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',   ##    monitoring validation accuracy 
    mode='max',               ##
    save_best_only=True)


In [20]:
hist=model.fit(X_train,Y_train,validation_data=(X_test, Y_test),validation_batch_size=16,
                batch_size=16, epochs=3, 
                callbacks=[model_checkpoint_callback])

Epoch 1/3
1033/1431 [====================>.........] - ETA: 16:13 - loss: 0.4899 - accuracy: 0.7620

KeyboardInterrupt: 

In [ ]:
#load saved model weights
model.load_weights('model/')

In [ ]:
Y_pred=model.predict(X_test)


In [ ]:
Y_pred=(Y_pred>0.5)*1.0

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_pred,Y_test))